In [27]:
import streamlit as st
import pandas as pd
from datetime import datetime # Para pegar a data atual
from PIL import Image # Imagens da tn e jpn
from graficos.Tribuna_do_Norte import tnPortal
from graficos.Tribuna_do_Norte import tnImpresso
import pandas as pd
import streamlit as st
from graficos.Tribuna_do_Norte import tnPortal

# Cria um espaço reservado vazio onde vai receber as imagens
image_placeholder = st.empty()

# Caminho das imagens
TN_image_path = Image.open("imagens/tribunaLogo.png") 
JPN_image_path = Image.open("imagens/jpnnatalLogo3.png") 

# Criar seletor 1 na coluna à esquerda
options1 = ["Escolha uma opção","Tribuna do norte", "JP News - Natal"]
selected_option1 = st.sidebar.selectbox("Selecione a opção 1:", options1)

# if selected_option1 == "Tribuna do norte":
#     image_placeholder.image(TN_image_path, use_column_width=True)
# elif selected_option1 == "JP News - Natal":
#     image_placeholder.image(JPN_image_path, use_column_width=True)

# Criar seletor 2 na coluna à esquerda
if selected_option1 == "Tribuna do norte":
    options2 = ["Site/Portal", "Google Analytics (portal)","Impresso","Instagram", "Facebook", "Twitter", "YouTube"]
    selected_option2 = st.sidebar.selectbox("Selecione a opção 2:", options2)
# Criar seletor 2 na coluna à esquerda
elif selected_option1 == "JP News - Natal":
    options2 = ["Instagram", "Twitter", "YouTube"]
    selected_option2 = st.sidebar.selectbox("Selecione a opção 2:", options2)
else:
    options2 = []
    selected_option2 = st.sidebar.selectbox("Selecione a opção 2:", options2)
    
# Definindo a data mínima e máxima
data_minima = pd.to_datetime('2023-01-01')
data_maxima = pd.to_datetime('2024-03-09')
#data_maxima = pd.to_datetime(tnPortal.df_noticias.iloc[-1]['not_datapub'])
    
# Adicionar seletor de períodos na coluna à esquerda
start_date = st.sidebar.date_input("Data de início", data_minima, min_value = data_minima, max_value = data_maxima, format="DD-MM-YYYY")

end_date = st.sidebar.date_input("Data de término", data_maxima, min_value = data_minima, max_value = data_maxima, format="DD-MM-YYYY") #+ pd.DateOffset(days=1) "DD-MM-YYYY"

# st.sidebar.write("AVISO (impresso): O dia 31/12/2023 não está sendo reconhecido corretamente. Não o selecionem, por favor.")
st.sidebar.write("AVISO (TN - online/Portal): No momento, por conta da mudança para o novo site, os dados disponíveis vão somente até 16/10/2023. Em breve os dados serão atualizados.")
st.sidebar.write("• Coisas a serem corrigidas (TN - impresso):")
st.sidebar.write("1. As datas de 2024 estão sendo interpretadas como se fossem em 2023, pontanto, alguns dados de 2024 estão sendo incluidos quando é selecionada alguma data do período de 01/01/2024 até 09/03/2024;")
st.sidebar.write("2. Valores individuais (no gráfico) de cada tópico de 'Notícias por editoria' não estão sendo filtrados corretamente de acordo com o período delecionado, e estão mostrando sempre seus valores totais. O valor total de todas as notícias juntas, a esquerda do gráfico e logo abaixo dos tópicos do gráfico, está correto exceto pelo erro citado no tópico 1;")
st.sidebar.write("Os erros a serem corrigidos citados acima passaram a ocorre por conta da entrada dos dados de 2024.")

start_date = start_date.strftime('%d-%m-%y') #'%d-%m-%Y'
end_date = end_date.strftime('%d-%m-%y')

# tnPortal.filtroDeDatas(start_date, end_date) é a função que recebe as datas de inicio e fim do período selecionado e atualiza os dfs
# retorna multiplos dfs que seram utilizados nas funções dos gráficos do portal
df_NOTICIAS_filtrado, editoria_freq, reporter_freq, reporter_unique, merge_ids_rep_noticias_editoria, fotografos, fotografos_teste = tnPortal.filtroDeDatas(start_date, end_date)

noticias_edi_somado, df_NOTICIAS_impresso_filtrado, reporteres_impresso, noticias_edi_somado = tnImpresso.filtroDeDatasImpresso(start_date, end_date)

# Raios para os gráficos de rosca
raio_interno = 0.7
raio_half = 0.2

'''RECEBENDO OS DF'''
# Recebe a tabela com as notícias do impresso
# low_memory=False por a tabela ser grande
df_noticias_impresso = pd.read_csv('tabelas/impresso/dados_impresso.csv', low_memory=False)
df_noticias_impresso['data'] = pd.to_datetime(df_noticias_impresso['data'])

# Recebe a tabela com as notícias do impresso
# low_memory=False por a tabela ser grande
df_editorias_impresso = pd.read_csv('tabelas/impresso/EDI_impresso.csv', low_memory=False)
df_editorias_impresso['data'] = pd.to_datetime(df_editorias_impresso['data'])


'''
MANIPULANDO OS DFs
'''
def filtroDeDatasImpresso(start_date, end_date):
    
    df_noticias_impresso_FILTRADAS = df_noticias_impresso.copy()

    df_noticias_impresso_FILTRADAS['data'] = pd.to_datetime(df_noticias_impresso_FILTRADAS['data']).dt.strftime('%m-%d-%y')

    df_NOTICIAS_impresso_filtrado = df_noticias_impresso_FILTRADAS.loc[(df_noticias_impresso_FILTRADAS['data'] > start_date) & (df_noticias_impresso_FILTRADAS['data'] < end_date)]

    # Altera o formato da data para '%d-%m-%y' após o filtro
    df_NOTICIAS_impresso_filtrado['data'] = pd.to_datetime(df_NOTICIAS_impresso_filtrado['data'], format='mixed').dt.strftime('%d-%m-%y')
    
    
    
    df_editorias_impresso_FILTRADAS = df_editorias_impresso.copy()

    df_editorias_impresso_FILTRADAS['data'] = pd.to_datetime(df_editorias_impresso_FILTRADAS['data']).dt.strftime('%m-%d-%y')

    df_editorias_impresso_filtrado = df_editorias_impresso_FILTRADAS.loc[(df_editorias_impresso_FILTRADAS['data'] > start_date) & (df_editorias_impresso_FILTRADAS['data'] < end_date)]

    # Altera o formato da data para '%d-%m-%y' após o filtro
    df_editorias_impresso_filtrado['data'] = pd.to_datetime(df_editorias_impresso_filtrado['data'], format='mixed').dt.strftime('%d-%m-%y')
    
    
    
    # Recebe o Series df_noticias_impresso['reporter_fotografo'] do df
    # value_counts().reset_index() conta a freqeuncia de cada informação na coluna de reporter_fotografo e organiza em ordem decrescente de acordo com a coluna que conta a freqência de cada informação
    # .replace() renomea linhas da coluna reporter_fotografo
    reporteres_impresso = df_NOTICIAS_impresso_filtrado['reporter_fotografo'].value_counts().reset_index().replace({'icarocesarcarvalho':'Icaro Cesar Carvalho', 'Fernandasouzajh': 'Fernanda Souza', 'adenilson_costa': 'Adenilson Costa'})

    # Renomeando as colunas por conta do .reset_index().replace que criou uma nova com as contagens de cada linha de reporter_fotografo
    reporteres_impresso.columns = ['reporter_fotografo', 'freq']

    # Recebe a coluna 'editoria' do df df_editorias_impresso
    noticias_edi = df_editorias_impresso_filtrado[['editoria', 'freq_edi']]

    # Nesse caso vai sem ['editoria'] por ser um tipo Series e só ter aquela coluna
    noticias_edi_uniRow = noticias_edi.drop_duplicates()

    # Agrupa o DataFrame pelo valor da coluna 'editoria' e soma os valores em cada grupo
    noticias_edi_somado = noticias_edi.drop_duplicates().groupby('editoria')['freq_edi'].sum().reset_index()

    # Foi feita da forma acima por a API do Trello já tinha a contagem de cada editoria disponível e, nesse caso, o calor não estaria correto se fosse usado .value_counts()

    # conta as aparições da linha Pauta sem editoria em editoria
    if 'Pauta sem editoria' in df_editorias_impresso_filtrado['editoria'].values:
        freq_pauta_sem_editoria = df_editorias_impresso_filtrado['editoria'].value_counts()['Pauta sem editoria']
    else:
        freq_pauta_sem_editoria = 0  # Ou qualquer valor padrão desejado

    #freq_pauta_sem_editoria = df_editorias_impresso_filtrado['editoria'].value_counts()['Pauta sem editoria']

    # Adiciona uma nova linha com com a contagem de Pauta sem editoria
    noticias_edi_somado.loc[len(noticias_edi_somado)] = ['Pautas sem editorias', freq_pauta_sem_editoria]

    # Organiza em ordem decrescente de acordo com a coluna freq_edi
    noticias_edi_somado = noticias_edi_somado.sort_values(by= 'freq_edi',ascending=False)
    
    #remove a ultima linha
    noticias_edi_somado = noticias_edi_somado[:len(noticias_edi_somado)-1]
    
    return noticias_edi_somado, df_NOTICIAS_impresso_filtrado, reporteres_impresso, noticias_edi_somado


c:\Users\Usuario\Documents\Repositórios\Dashboard-TN\graficos\Tribuna_do_Norte\tnPortal.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_NOTICIAS_filtrado['not_datapub'] = pd.to_datetime(df_NOTICIAS_filtrado['not_datapub'], format='mixed').dt.strftime('%d-%m-%y')
c:\Users\Usuario\Documents\Repositórios\Dashboard-TN\graficos\Tribuna_do_Norte\tnImpresso.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_NOTICIAS_impresso_filtrado['data'] = pd.to_datetime(df_NOTICIAS_impresso_filtrado['data'], form

In [31]:
df_noticias_impresso['data']

df_editorias_impresso['data'] = pd.to_datetime(df_editorias_impresso['data'])
type(df_editorias_impresso['data'][1])
#df_editorias_impresso['data'][1]

df_noticias_impresso_FILTRADAS = df_noticias_impresso.copy()

#df_noticias_impresso_FILTRADAS['data'] = pd.to_datetime(df_noticias_impresso_FILTRADAS['data']).dt.strftime('%m-%d-%y')

df_NOTICIAS_impresso_filtrado = df_noticias_impresso_FILTRADAS.loc[(df_noticias_impresso_FILTRADAS['data'] > start_date) & (df_noticias_impresso_FILTRADAS['data'] < end_date)]

# Altera o formato da data para '%d-%m-%y' após o filtro
#df_NOTICIAS_impresso_filtrado['data'] = pd.to_datetime(df_NOTICIAS_impresso_filtrado['data'], format='mixed').dt.strftime('%d-%m-%y')

type(df_NOTICIAS_impresso_filtrado['data'][1])

pandas._libs.tslibs.timestamps.Timestamp